In [171]:
# install required packages
import pandas as pd
import numpy as np
from utils.database import query_from_file
import datetime
import time
import plotly.express as px
path = '/Users/jennamiles/Documents/credentials'
import psutil
from datetime import datetime, timedelta

In [2]:
#this is from database.py
#need to update in utils because of directory issue
import os
from psycopg2 import connect
from pandas import read_sql

def query(sql_statement: str, **kwargs):
    with open(f'{path}', 'r') as credentials:
        connection = connect(credentials.read())
        cursor = connection.cursor()

        if len(kwargs) > 0:
            dataframe = read_sql(sql_statement.format(**kwargs), connection)
        else:
            dataframe = read_sql(sql_statement, connection)

        assert len(dataframe) > 0, 'query returned no results'
        return dataframe
    
def query_from_file(sql_file: str, **kwargs):
    with open(sql_file, 'r') as f:
        return query(f.read(), **kwargs)

In [48]:
#read in pure data from SQL query:\# read in the starling transacitons 
start = time.time()
df = query_from_file('pure_txns.sql')
end = time.time()
query_time = end-start
print(f"Query took {query_time:.2f} seconds")

Query took 2.52 seconds


In [49]:
#additional date columns that will be useful 
df['date'] = df['updated_date'].dt.date
df['day'] = df['updated_date'].dt.isocalendar().day
df['week'] = df['updated_date'].dt.isocalendar().week
df['week_start'] = (df['updated_date'] - pd.TimedeltaIndex(df['updated_date'].dt.dayofweek, unit='D')).dt.date
df['month'] = df['updated_date'].dt.month

In [50]:
#other amendments needed
df['price'] = pd.to_numeric(df['item_price'])

In [51]:
#make a receipt count on all Pure transactions so we can tell if the offer txn was first with retailer
#warning: only can tell if first using flux 
df_receipts = df[['customer_id','receipt_id','total_amount','updated_date','month','location_id']].drop_duplicates().sort_values(['customer_id','updated_date'])
df_receipts['count'] = df_receipts.groupby(['customer_id'])['receipt_id'].cumcount()+1

In [52]:
#offer transactions
offer_txns = df_receipts[df_receipts['price']<0].copy()

In [53]:
#pick an offer to use with highest volume
offer_txns.groupby(['name']).nunique().sort_values(by = 'customer_id', ascending = False)['customer_id'].head(3)

name
Free Coffee YELLOW                    94
Eat Out to Help Out discount for 1    16
Eat Out to Help Out discount for 2     5
Name: customer_id, dtype: int64

### Chosen offer = Free Coffe YELLOW

In [88]:
#pick the most used as example
df_off = offer_txns[offer_txns['name'] == 'Free Coffee YELLOW'].copy()

In [89]:
#find only first transactions at Pure:
df_receipts['new_segment'] = np.where(df_receipts['count']==1,'new','existing')

In [90]:
# using the first transactions data
# flag in offer txns if customers were new or existing:
df_off = pd.merge(df_off,df_receipts[['customer_id','new_segment','receipt_id']], on = ['customer_id','receipt_id'],how = 'inner')

In [117]:
#how many new vs existing customers used the offer?
total_cust = df_off['customer_id'].nunique()

In [92]:
new_existing = pd.DataFrame(df_off.groupby('new_segment').nunique()['customer_id']).reset_index()
new_existing['pcent'] = (new_existing['customer_id']/total)*100

In [93]:
new_existing

,new_segment,customer_id,pcent
0,existing,80,85.106383
1,new,17,18.085106


In [120]:
#get the baseline % of customers that are new over a month 
#might do more advanced version of who is the baseline

#get new customers per month
total = df_receipts.groupby(['month','new_segment']).nunique()['customer_id'].unstack().sum(axis = 1)

#pcent
monthly_new_baseline = ((df_receipts.groupby(['month','new_segment']).nunique()['customer_id'].unstack()['new']/total)*100).mean()

In [111]:
df_receipts.groupby(['month','new_segment']).nunique()['customer_id'].unstack()

new_segment,existing,new
month,,
9,242,874
10,357,628
11,103,112
12,3,21


## Metrics for customer using offer

In [119]:
print(f'Total number of customers on the offer is {total_cust}')
print('\n')
print(new_existing)
new_pcent = new_existing[new_existing['new_segment']=='new'].pcent.values[0]
diff_new_pcent = new_pcent - monthly_new_baseline
print('\n')
print('\n')
print(f'The average new customer % per month is {monthly_new_baseline:.2f}%')
print(f'The difference to baseline in % new customers is {diff_new_pcent:.2f}%')

Total number of customers on the offer is 94


  new_segment  customer_id      pcent
0    existing           80  85.106383
1         new           17  18.085106




The average new customer % per month is 70.42%
The difference to baseline in % new customers is -52.33%


## Metrics for frequency of redeeming group

In [174]:
# frequency 
# redeemers vs non redeemers: DONE 
# new vs existing 
import datetime

#identify those that are redeemers
off_custs = df_off['customer_id'].unique()

#apply as segmentation onto df_receipts
df_receipts['redeemer_type'] = np.where(df_receipts['customer_id'].isin(off_custs),'redeemer','non-redeemer')

#restrict to the offer live period
date_min = df_off['date'].min()
#date_max = df_off['date'].max()

#or can set a number of days:
d = 30
date_max = date_min + datetime.timedelta(days=d)
df_post = df_receipts[(df_receipts['updated_date'].dt.date >= date_min) & (df_receipts['updated_date'].dt.date <= date_max)]

#change above to depend on how many days since customers first transaction - so we don't count the transactions before they used the offer?

#average number of transactions by group, AFTER the start of the offer
ATF = df_post.groupby('redeemer_type').nunique()['receipt_id']/df_post.groupby('redeemer_type').nunique()['customer_id']

print(f'Offer start date is {date_min}')
print(f'Offer end date is {date_max}')
print('\n')
print('ATF during offer period is:\n',ATF)

Offer start date is 2020-09-08
Offer end date is 2020-10-08


ATF during offer period is:
 redeemer_type
non-redeemer    1.506977
redeemer        4.679487
dtype: float64


In [160]:
df_off.head()

,customer_id,account_id,receipt_id,item_id,item_price,updated_date,location_id,total_amount,sku,name,date,day,week,week_start,month,price,new_segment
0,a563183d-0b0c-43cb-9e7f-c63941b1386c,222b3a93-1906-4dcf-8856-ec674fcea612,00d6eb35-e423-43fc-9671-90095953643b,e464c7e9-c24d-4508-a5a2-14201a04f156,-235,2020-10-02 09:40:07.563567+00:00,1190fc2f-bde5-4fc7-8b07-73ff31115a77,1003.0,158,Free Coffee YELLOW,2020-10-02,5,40,2020-09-28,10,-235,existing
1,66d5602c-e532-47eb-99a1-7686a60e0970,ec0ce942-5776-41ec-b09d-84bdc0df82b4,087ef9a5-9144-4488-9e9a-7085ecdb0d9a,e464c7e9-c24d-4508-a5a2-14201a04f156,-299,2020-09-23 05:11:38.263706+00:00,ffe8b99e-70b2-47fb-a456-eda6d6d67a16,455.0,158,Free Coffee YELLOW,2020-09-23,3,39,2020-09-21,9,-299,existing
2,403eba24-cf50-4853-be09-2f979069f8bc,ca4c4ea9-6e62-496f-8f7d-6d12914b0043,09655cf2-31e6-4e49-9509-753c3b2d9d49,e464c7e9-c24d-4508-a5a2-14201a04f156,-299,2020-10-07 10:42:37.459221+00:00,9aada63a-130d-4d37-839a-bfd644eac5d5,285.0,158,Free Coffee YELLOW,2020-10-07,3,41,2020-10-05,10,-299,existing
3,69dcebd3-3f29-4fd5-9535-0930d9035e3a,354c868d-9a98-4f4c-9588-98089f6402e4,0d3a5bf2-6933-47be-8ab5-f0acf6dbdea3,e464c7e9-c24d-4508-a5a2-14201a04f156,-235,2020-10-14 08:49:23.481019+00:00,9aada63a-130d-4d37-839a-bfd644eac5d5,299.0,158,Free Coffee YELLOW,2020-10-14,3,42,2020-10-12,10,-235,existing
4,fba8adae-e86b-4534-9446-56701b589b7d,eb5fb869-6eb1-4703-80cf-f3fda8e546cf,10cd5074-17c5-41ca-a421-34c12ad96ea4,e464c7e9-c24d-4508-a5a2-14201a04f156,-299,2020-10-15 08:34:18.869389+00:00,d1f67703-2710-484e-acf0-3f56a74db824,255.0,158,Free Coffee YELLOW,2020-10-15,4,42,2020-10-12,10,-299,existing


# Results

## Metrics

***Offer just started***:

- customers using offer
    - total number :Done
    - new -  % :Done
    - existing - % :Done
    - vs baseline (e.g how many new customers do you get in a typical <period of time>) :Done (used month)
- Frequency of the redeeming group over customer type (new vs existing, how many times do they come)
    - Overall frequency
    - vs non-redeeming group
- % of new customers that have purchased again
- Revenue impact
    - Total
    - per customer
    - Value of discount
        - per customer
- ATV of the redeeming group over customer type (new & existing)
    - Overall ATV
    - vs non-redeeming group
- Basket size
- Time of day / day of week of redemption
    - New vs existing
- redemptions on the offer (if we've seen a customer redeem more than once - otherwise assume offers are single redemption per customer)
    - new
    - existing
    - by timeframe (e.g. day, week, month depending on length of offer)
- ~~Redemptions per location? (maybe out of scope)~~

***After x time***:

- Existing customers that fall into one of segments? E.G. are they changing groups (spending more)

***Post offer analysis***:

- Long-term impact on frequency of customers redeeming the offer
    - vs non-redeemers
- Long term impact of ATV of customers redeeming the offer
    - vs non-redeemers
- Retention of customers who have redeemed the offer vs baseline
- Segment of customers acquired via the offer
    - % in each segment vs expected %
- ~~Value of the customers acquired via the offer~~